In [1]:
import pandas as pd
df=pd.read_csv("C:/Users/DELL/Desktop/csv files/IMDB Dataset.csv")

In [2]:
df.head(10)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [3]:
df['review'][10]

'Phil the Alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines.<br /><br />At first it was very odd and pretty funny but as the movie progressed I didn\'t find the jokes or oddness funny anymore.<br /><br />Its a low budget film (thats never a problem in itself), there were some pretty interesting characters, but eventually I just lost interest.<br /><br />I imagine this film would appeal to a stoner who is currently partaking.<br /><br />For something similar but better try "Brother from another planet"'

In [4]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count=CountVectorizer()
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'])
bag=count.fit_transform(docs)
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [5]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


# Task 3 Term Frequency and Inverse Document Frequency
#Term frequencies alone do not contribute to distinct information
#tf-idf (t,d) = tf(t,d) X idf(t,d)
#idf(t,d) = log(nd / 1 + df(d,t))
#where nd = total number of documents and df(d,t) = number of documents that contain the term t 

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True,smooth_idf=True)


In [7]:


print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.         0.43370786 0.         0.55847784 0.55847784 0.
  0.43370786 0.         0.        ]
 [0.         0.43370786 0.         0.         0.         0.55847784
  0.43370786 0.         0.55847784]
 [0.50238645 0.44507629 0.50238645 0.19103892 0.19103892 0.19103892
  0.29671753 0.25119322 0.19103892]]


In [8]:
from nltk.stem.porter import PorterStemmer
porter=PorterStemmer()
def stemmer_tokenize(text):
    return[porter.stem(word) for word in text.split()]
stemmer_tokenize('coders like coding and thus they code')

['coder', 'like', 'code', 'and', 'thu', 'they', 'code']

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None,lowercase=False,tokenizer=stemmer_tokenize,use_idf=True,smooth_idf=True)
Y=df.sentiment.values
X=tfidf.fit_transform(df.review)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    X,Y,random_state=1, train_size = 0.5,shuffle=False
)
import pickle
from sklearn.linear_model import LogisticRegressionCV
clf=LogisticRegressionCV(cv=5,scoring='accuracy',random_state=0,n_jobs=-3,verbose=3,max_iter=300).fit(X_train,Y_train)
saved_model=open('saved_model.sav','wb')
pickle.dump(clf,saved_model)
saved_model.close()

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-3)]: Done   2 out of   5 | elapsed:  3.1min remaining:  4.7min
[Parallel(n_jobs=-3)]: Done   5 out of   5 | elapsed:  3.3min finished


In [13]:
filename='saved_model.sav'
saved_clf=pickle.load(open(filename,'rb'))
saved_clf.score(X_test,Y_test)

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:2260: ChangedBehaviorWarning: The long-standing behavior to use the accuracy score has changed. The scoring parameter is now used. This warning will disappear in version 0.22.
  ChangedBehaviorWarning)


0.8898